In [2]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np
import re

# print(data.columns)
# Extract 'problems' and 'findings' columns
df= pd.read_csv("C:\FYP DATASET\indiana_reports.csv")[["uid","findings"]]
#print df train element by element
print(df.iloc[0])




uid                                                         1
findings    The cardiac silhouette and mediastinum size ar...
Name: 0, dtype: object


In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(device)
#Also print gpu name
if device == 'cuda':
    print(cuda.get_device_name(0))

In [4]:
# Sample 20% of the data randomly without replacement
sample_df = df.sample(frac=0.2, random_state=42)  # random_state ensures reproducibility

# If you need to reset the index of the sampled DataFrame
sample_df.reset_index(drop=True, inplace=True)

print(sample_df)

In [8]:
import pandas as pd

# Load the sampled DataFrame (assuming you have already done this)
sample_df = pd.read_csv("C:/FYP DATASET/indiana_reports.csv")

# Load the projections data
projections_df = pd.read_csv("C:/FYP DATASET/indiana_projections.csv")

# Initialize an empty list to store (filename, finding) tuples
filename_finding_pairs = []

# Iterate over each row in your sampled DataFrame
for index, row in sample_df.iterrows():
    uid = row['uid']
    finding = row['findings']
    
    # Find rows in projections_df where 'uid' matches
    matched_filenames = projections_df[projections_df['uid'] == uid]['filename'].tolist()
    
    # For each filename matched, create a (filename, finding) tuple and append to the list
    for filename in matched_filenames:
        filename_finding_pairs.append((filename, finding))

# Print the first few pairs to verify
print(filename_finding_pairs[:10])  # Adjust as needed


[('1_IM-0001-4001.dcm.png', 'The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.'), ('1_IM-0001-3001.dcm.png', 'The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.'), ('2_IM-0652-1001.dcm.png', 'Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX.'), ('2_IM-0652-2001.dcm.png', 'Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX.'), ('3_IM-1384-1001.dcm.png', nan), ('3_IM-1384-2001.dcm.png', nan), ('4_IM-2050-1001.dcm.png', 'There are diffuse bilateral interstitial and alveolar opacities consistent with chronic obstructive lung disease and bullous 

In [19]:
import os
import numpy as np
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from nltk.translate.bleu_score import sentence_bleu
import create_model as cm  # Ensure this is your module for loading models and such

def load_model():
    model_directory = "/FYP_DATASET/results/"
    loaded_model = AutoModelForCausalLM.from_pretrained(model_directory)
    tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    loaded_model = loaded_model.to(device)
    return loaded_model, tokenizer, device

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    # Perform any required preprocessing here
    return np.array(image)

def predict(image_1, image_2, model, tokenizer, device):
    # Assuming `cm.function1` is your method to get captions from images, adjust as needed.
    caption = cm.function1([image_1], [image_2 if image_2 is not None else image_1], model)

    input_ids = tokenizer.encode(caption[0], return_tensors="pt").to(device)
    output = model.generate(input_ids, max_length=200, pad_token_id=tokenizer.eos_token_id, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.0)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    return decoded_output

def calculate_bleu_score(reference_text, candidate_text):
    reference = [reference_text.split()]
    candidate = candidate_text.split()
    score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))  # You can adjust weights
    return score

def evaluate_bleu_scores(filename_finding_pairs, image_directory):
    model, tokenizer, device = load_model()
    scores = []
    
    for image_name, actual_report in filename_finding_pairs:
        print(image_name)
        print(actual_report)
        image_path = os.path.join(image_directory,image_name)  # Adjust format as needed
        if not os.path.exists(image_path):
            print(image_path)
            print(f"Image {uid} not found.")
            continue
        
        image_1 = preprocess_image(image_path)
        generated_report = predict(image_1, None, model, tokenizer, device)
        bleu_score = calculate_bleu_score(actual_report, generated_report)
        scores.append(bleu_score)
        print(f"BLEU score for {uid}: {bleu_score}")

    average_bleu_score = np.mean(scores)
    print(f"Average BLEU Score: {average_bleu_score}")

# Example usage
# filename_finding_pairs = [
#     ("image1", "The chest X-ray shows no acute disease."),
#     ("image2", "There is evidence of a consolidation in the right lower lobe suggesting pneumonia.")
# ]
image_directory = "\images\images_normalized"  # Update this to your images' directory

evaluate_bleu_scores(filename_finding_pairs, image_directory)


image1
The chest X-ray shows no acute disease.
\images\images_normalized\image1
Image 3999 not found.
image2
There is evidence of a consolidation in the right lower lobe suggesting pneumonia.
\images\images_normalized\image2
Image 3999 not found.
Average BLEU Score: nan


c:\Users\Saad Subhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Saad Subhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [2]:
import pandas as pd

def get_report_by_uid(uid, csv_file_path='indiana_reports.csv'):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Convert uid to int to match the DataFrame's format if necessary
    uid_int = int(uid)
    
    # Find the row with the matching uid
    report_row = df[df['uid'] == uid_int]
    
    # If a matching report is found, extract the findings
    if not report_row.empty:
        findings = report_row.iloc[0]['findings']
        print(f"Findings for UID {uid}: {findings}")
        return findings
    else:
        print(f"No findings found for UID {uid}")
        return None

# Example usage
uid = 10  # Example UID extracted earlier
csv_file_path = "../indiana_reports.csv"  # Adjust this path as necessary
findings = get_report_by_uid(uid, csv_file_path)

Findings for UID 10: The cardiomediastinal silhouette is within normal limits for size and contour. The lungs are normally inflated without evidence of focal airspace disease, pleural effusion, or pneumothorax. Stable calcified granuloma within the right upper lung. No acute bone abnormality..
